In [1]:
import pandas as pd

df =pd.read_csv('D:\\AI-youtube\\ai-youtube-search\data_files\\videos_info\\playlist_info\\Dataset.csv')
df.shape

(90633, 7)

In [2]:
df['text']


0         Gonna start this stack quest with the silly s...
1         Note, this stack quest was originally prepare...
2         If you like silly songs, are you interested i...
3         Why not why? If you don't like silly songs, a...
4         Otherwise, you might not like Stack Quest. Wh...
                               ...                        
90628    already taken Sorry. I also by the way, also c...
90629    It has a bunch of parts that are maybe not sup...
90630    might be a bit tricky. And, but the rest and t...
90631    happens, maybe there's a CNN, maybe the CNN fe...
90632    give this thing a read, give this video a like...
Name: text, Length: 90633, dtype: object

In [7]:
conda install -c pytorch/label/nightly faiss-cpu

Retrieving notices: ...working... done
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 23.3.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.3.1




In [8]:
import faiss

In [3]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
from sentence_transformers import SentenceTransformer

#Create a large list of 100k sentences
sentences = df['text']

model = SentenceTransformer('all-MiniLM-L6-v2')
#Start the multi-process pool on all available CUDA devices
pool = model.start_multi_process_pool(['cpu','cpu','cpu'])

#Compute the embeddings using the multi-process pool
final_emb = model.encode_multi_process(sentences, pool)
print("Embeddings computed. Shape:", final_emb.shape)

#Optional: Stop the proccesses in the pool
model.stop_multi_process_pool(pool)


Embeddings computed. Shape: (90633, 384)


In [4]:
len(final_emb)

90633

In [6]:
final_emb.shape

(90633, 384)

In [5]:
d=final_emb.shape[1]
d

384

In [9]:
index=faiss.IndexFlatL2(d)

In [10]:
index.is_trained

True

In [11]:
index.add(final_emb)

In [12]:
index.ntotal

90633

In [13]:
xq=model.encode(['what is bootstrap'])

In [14]:
%%time
D, I = index.search(xq,5)
print(I)
print(D)

[[48901 30687 30686 14421  1413]]
[[0.7749672  0.8113897  0.83024615 0.9622795  0.99349165]]
CPU times: total: 46.9 ms
Wall time: 1.52 s


In [15]:
lines=df['text']

In [16]:
[f'{i}: {lines[i]}' for i in I[0]]

["48901: and stuff like that I'll show you exactly the process I go through. So Bootstrap has a website and you go there and you can kind of look at their examples of things that you can do with Bootstrap and then you just basically copy and paste and kind of edit and change. So it's actually a lot easier than it might seem. So with that let's go ahead and get started.",
 "30687: So we have some cool examples here so this is one card we can go down a little bit more or this one I think this is probably the one I'll use. Okay so basically Bootstrap is like a framework which is pretty lightweight and you have all these really nice examples of how you can build something that was super nice. So what we can do is actually just copy this, we'll go back over to our code and what I'm going to do is I'm going to create the definition for one of these cards.",
 "30686: So if I just type in Bootstrap cards, you can see it's in my history but I'm going to just put Bootstrap cards for now so you c

In [17]:
xq2=model.encode(['what is cross validation'])

In [18]:
%%time
D2, I2 = index.search(xq2,5)
print(I2)
print(D2)

[[38113 38111 38074 38068    28]]
[[0.6957379  0.7749776  0.8071182  0.83150375 0.84595877]]
CPU times: total: 62.5 ms
Wall time: 62.4 ms


In [19]:
[f'{i}: {lines[i]}' for i in I2[0]]

["38113: to explain cross validation and how I understand it I would I would look into it into it yourself as another piece of homework but if you imagine if you imagine let's let's do this let's use a piece of paper while cat boost is running we've got a little countdown here so you've got some time to play with imagine you have your data here right just as one simple piece of paper right and you do one your model goes down and it fits all the rows in one hit okay it might have done a really good job right because some of what it does is randomly initialized what does that mean so",
 "38111: what it does and now I actually can't couldn't explain to you what what it does on the inside but just in my experience of using it multiple times for multiple projects it does take a little bit to run because I think I think it's doing a combination of different types of models so it's not just doing one one one single way of fitting on the data it's doing it's doing multiple different ways so it

In [36]:
nlist=10
quantizer=faiss.IndexFlatL2(d)
index=faiss.IndexIVFFlat(quantizer,d,nlist)

In [37]:
index.is_trained

False

In [38]:
index.train(embeddings)

In [39]:
index.add(embeddings)

In [40]:
index.ntotal

32

In [41]:
%%time
D, I =index.search(xq,5)
print(I)

[[ 5 13 12 11  4]]
CPU times: total: 0 ns
Wall time: 3 ms


In [42]:
[f'{i}: {lines[i]}' for i in I[0]]

["5:  To create a bootstrap dataset that is the same size as the original, we just randomly select samples from the original dataset. The important detail is that we're allowed to pick the same sample more than once. This is the first sample that we randomly select.",
 "13:  Using a bootstrap sample and considering only a subset of variables at each step results in a wide variety of trees. The variety is what makes random forests more effective than individual decision trees. Sweet. Now that we've created a random forest, how do we use it?",
 '12:  Now, go back to step one and repeat. Make a new bootstrap dataset and build a tree considering a subset of variables at each step. Ideally, you do this hundreds of times, but we only have space to show six. But you get the idea.',
 "11:  And we just build the tree as usual, but only considering a random subset of variables at each step. Double bound. We built a tree, one, using a bootstrap dataset, and two, only considering a random subset o

In [49]:
m=8
bits=5

quantizer=faiss.IndexFlatL2(d)
index=faiss.IndexIVFPQ(quantizer,d,16,m,bits)

In [50]:
index.is_trained

False

In [51]:
index.train(embeddings)

In [52]:
index.add(embeddings)

In [53]:
%%time
D, I =index.search(xq,5)
print(I)

[[ 5  7 12 -1 -1]]
CPU times: total: 31.2 ms
Wall time: 1.26 s


In [3]:
import pandas as pd
import faiss

In [2]:
df1=pd.read_csv('data_set.csv')
df1.shape

(128, 6)

In [4]:
from sentence_transformers import SentenceTransformer

#Create a large list of 100k sentences
sentences = df1['text']

model = SentenceTransformer('all-MiniLM-L6-v2')
#Start the multi-process pool on all available CUDA devices
pool = model.start_multi_process_pool(['cpu','cpu','cpu'])

#Compute the embeddings using the multi-process pool
final_emb = model.encode_multi_process(sentences, pool)
print("Embeddings computed. Shape:", final_emb.shape)

#Optional: Stop the proccesses in the pool
model.stop_multi_process_pool(pool)

Embeddings computed. Shape: (128, 384)


In [5]:
d=final_emb.shape[1]
d

384

In [6]:
index=faiss.IndexFlatL2(d)

In [7]:
index.is_trained

True

In [8]:
index.ntotal

0

In [9]:
index.add(final_emb)

In [10]:
xq=model.encode(['what is bootstrap'])

In [11]:
%%time
D, I =index.search(xq,5)
print(I)

[[32 20 26 24 28]]
CPU times: total: 0 ns
Wall time: 712 ms


In [33]:
s=df1.loc[I[0],['url','start']]

In [40]:
url=df1['url']
start=df1['start']
l=[df1['url'][i]+'?t='+str(df1['start'][i]) for i in I[0]]
l[1]



'https://www.youtube.com/watch?v=J4Wdy0Wc_xQ?t=91.76'

In [21]:
str(df1['start'][I[0]])

'32    138.84\n20     91.76\n26    119.00\n24    109.76\n28    126.40\nName: start, dtype: float64'